In [9]:
! pip install mediapipe opencv-python
! pip install pyautogui


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 165.2 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 361.7 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import mediapipe as mp
from Helper_Function.gesture_maths import distance
import numpy as np
import pyautogui


In [4]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

In [12]:
cap = cv2.VideoCapture(0)
stop = False

# Get the screen size
screen_width, screen_height = pyautogui.size()


while not stop:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    height, width, _ = frame.shape

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]    

            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

            right_click_distance = distance(middle_finger_tip, thumb_tip)
            left_click_distance = distance(index_finger_tip, thumb_tip)

            # Set distance thresholds for clicking
            right_click_threshold = 0.05  
            left_click_threshold = 0.05 

            # Calculate cursor position based on palm keypoints
            palm_keypoints = ['THUMB_MCP', 'INDEX_FINGER_MCP', 'MIDDLE_FINGER_MCP', 'RING_FINGER_MCP', 'PINKY_MCP']
            x_coords = []
            y_coords = []

            for keypoint in palm_keypoints:
                landmark = getattr(mp_hands.HandLandmark, keypoint)
                x_coords.append(hand_landmarks.landmark[landmark].x * width)
                y_coords.append(hand_landmarks.landmark[landmark].y * height)

            cursor_point = {
                'x': int(np.mean(x_coords)),
                'y': int(np.mean(y_coords))
            }

            # Move the mouse cursor to the calculated point
            screen_x = int((cursor_point['x'] / width) * screen_width)
            screen_y = int((cursor_point['y'] / height) * screen_height)
            pyautogui.moveTo(screen_x, screen_y)

            # Clicking Logic
            if left_click_distance < left_click_threshold:
                pyautogui.click()  # Perform left click
                    

            # Right click Logic
            if right_click_distance < right_click_threshold:
                pyautogui.rightClick()  # Perform right click

    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        stop = True

cap.release()
cv2.destroyAllWindows()
